In [16]:
# import libraries
from bs4 import BeautifulSoup as bsoup
import urllib.request
import csv
import requests
import pandas as pd

In [17]:
# specify the url
urlpage = 'https://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'

In [18]:
#Check the robots file :
!curl  https://www.fasttrack.co.uk/robots.txt

User-agent: *
Disallow: /wp-admin/
Allow: /wp-admin/admin-ajax.php


In [19]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
page

HTTPError: HTTP Error 403: Forbidden

### Let's try this now

In [20]:
from urllib.request import Request, urlopen

#Make the request and open the url :
req = Request('https://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

#Create a beutiful soup object so we can work with the data
soup = bsoup(webpage, 'html.parser')

#Sanity check 
soup

<!-- Template Name: League Table page
-->
<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie ie6 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie ie7 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie ie8 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie ie9 no-js" lang="en-GB"> <![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" lang="en-GB">
<!--<![endif]-->
<!-- the "no-js" class is for Modernizr. -->
<head id="live2-fasttrack-com"><link href="https://3s171z3x2dmy48s75v3z1ha8-wpengine.netdna-ssl.com/wp-content/plugins/simple-download-monitor/css/sdm_wp_styles.css?ver=5.7" id="sdm-styles-css" media="all" rel="stylesheet" type="text/css"/><link href="https://3s171z3x2dmy48s75v3z1ha8-wpengine.netdna-ssl.com/wp-includes/css/dist/block-library/style.min.css?ver=5.7" id="wp-block-library-css" media="all" rel="stylesheet" type="text/css"/>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine (even in intranet) & 

In [22]:
# Let's find the data from the table :
table = soup.find('table', attrs={'class': 'tableSorter2'})
# table
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 101


In [23]:
#Let's look at the top 2 rows to check the structure of each row in our table :
results[:2]

[<tr>
 <th>Rank</th>
 <th>Company</th>
 <th class="">Location</th>
 <th class="no-word-wrap">Year end</th>
 <th class="" style="text-align:right;">Annual sales rise over 3 years</th>
 <th class="" style="text-align:right;">Latest sales £000s</th>
 <th class="" style="text-align:right;">Staff</th>
 <th class="">Comment</th>
 <!--				<th>FYE</th>-->
 </tr>, <tr>
 <td>1</td>
 <td><a href="https://www.fasttrack.co.uk/company_profile/revolut-3/"><span class="company-name">Revolut</span></a>Digital banking services provider</td>
 <td>East London</td>
 <td>Dec 19</td>
 <td style="text-align:right;">309.89%</td>
 <td style="text-align:right;">162,722</td>
 <td style="text-align:right;">1,289</td>
 <td>Launched in America on the same day that the UK locked down</td>
 <!--						<td>Dec 19</td>-->
 </tr>]

## Can save the data to a file and read it back in

In [ ]:
#We can of course save the data to a file and read it back in if we need to :
#Open file to write the results to :
# outF = open("techtrack100.csv", "w")

# # loop over results
# for result in results:
#     data = result.find_all('td')
    
#     #Verify that the table row is not empty
#     if len(data) == 0:
#         continue
    
#     outF.write(data)
#     outF.write("\n")
    
# outF.close()

#Read in contents from file :
# with open('techtrack100.csv', 'r') as file:
#     my_reader = csv.reader(file, delimiter=',')
#     for row in my_reader:
#         print(row)

## Let's extract some information out of the website and reorganize the data from the companies.

In [24]:
# create and write headers to a list 
rows = []
rows.append(['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'])
print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments']]


In [25]:
#Let's look at the data once again :
results

[<tr>
 <th>Rank</th>
 <th>Company</th>
 <th class="">Location</th>
 <th class="no-word-wrap">Year end</th>
 <th class="" style="text-align:right;">Annual sales rise over 3 years</th>
 <th class="" style="text-align:right;">Latest sales £000s</th>
 <th class="" style="text-align:right;">Staff</th>
 <th class="">Comment</th>
 <!--				<th>FYE</th>-->
 </tr>, <tr>
 <td>1</td>
 <td><a href="https://www.fasttrack.co.uk/company_profile/revolut-3/"><span class="company-name">Revolut</span></a>Digital banking services provider</td>
 <td>East London</td>
 <td>Dec 19</td>
 <td style="text-align:right;">309.89%</td>
 <td style="text-align:right;">162,722</td>
 <td style="text-align:right;">1,289</td>
 <td>Launched in America on the same day that the UK locked down</td>
 <!--						<td>Dec 19</td>-->
 </tr>, <tr>
 <td>2</td>
 <td><a href="https://www.fasttrack.co.uk/company_profile/football-index/"><span class="company-name">Football Index</span></a>Football trading platform</td>
 <td>Central London

## Time to put all of this to good use. 

## Let's now go through the data and extract the fields : company name, webpage and description while leaving all other attributes as they are

In [34]:
for result in results:
    #Pull data from row :
    data = result.find_all('td')
    
    #Deal with empty results :
    if len(data) == 0:
        continue
    
    #Sanity check :
    rank = data[0].get_text()
    thingToDo = data[1]
#     CompanyName = 
#     Webpage =
#     Description =
    Location = data[2].get_text()
    Yearend = data[3].get_text()
    Annual = data[4].get_text()
    Sales = data[5].get_text()
    Staff = data[6].get_text()
    Comments = data[7].get_text()
    
    print(thingToDo.find('a').get_text())
    
#     print(rank, Location, Yearend, Annual, Sales, Staff, Comments)
    input()
    
    

Revolut

Football Index

Tessian

Pollen


KeyboardInterrupt: 